In [1]:
### To create train dataset, we use already existing registered and unstained 1024 x 1024 tiles. The problem is, that the mask is generated by using the HE WSI, and b/c HE and unstained WSI are different, we need to somehow match the tiles of the mask and the registered, unstained tiles.

In [6]:
import numpy as np
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import cv2
from natsort import natsorted
import os
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
import xml.etree.ElementTree as ET
import pandas as pd
import skimage.measure
import scipy.stats as stats
import time
import shutil
from tqdm import tqdm
from skimage.measure import label, regionprops_table
from natsort import natsorted

In [11]:
OPENSLIDE_PATH = r'C:\Users\labuser\Downloads'

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [16]:
unstained_registered_tile_src = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\US"
label_mask_src = r"\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined"
label_mask_list = [x for x in os.listdir(label_mask_src) if x.endswith(".png")]
label_mask_list = [x for x in label_mask_list if x[:3] == "OTS"]
label_mask_list = label_mask_list[2:]
label_mask_list = natsorted(label_mask_list)
label_mask_name_list = [x.replace(".png","") for x in label_mask_list]
label_mask_src = [os.path.join(label_mask_src,x) for x in label_mask_list]
unstained_registered_tile_src = [os.path.join(unstained_registered_tile_src,x) for x in label_mask_list]

In [17]:
label_mask_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\1um\\classification_v9_combined\\OTS_14684_1_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\1um\\classification_v9_combined\\OTS_14684_2_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\1um\\classification_v9_combined\\OTS_14684_3_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\1um\\classification_v9_combined\\OTS_14684_5_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\1um\\classification_v9_combined\\OTS_14684_6_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\1um\\classification_v9_combined\\OTS_14684_7_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\1um\\classification_v9_combined\\OTS_14684_8_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\1um\\classification_v9_combined\\OTS_14684_9_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\1um\\classification_v9_combined\\OTS_14832_1_h

In [18]:
unstained_registered_tile_src

['\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_2_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_3_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_5_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_6_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_7_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_8_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_9_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14832_1_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14832_2_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14832_5_he.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\O